In [ ]:
# Importing Pandas for data manipulation
import pandas as pd
import json

def encode_metrics_to_separate_dataframes(file_path):
    """
    Reads a JSON file line by line and encodes loss and mAP metrics into separate DataFrames.
    
    Parameters:
        file_path (str): The path to the JSON file.
        
    Returns:
        DataFrame, DataFrame: Two DataFrames, one containing loss metrics and the other containing mAP metrics.
    """
    # Initialize empty dictionaries to hold the metrics
    loss_dict = {}
    mAP_dict = {}
    
    # Read the JSON file line by line
    with open(file_path, 'r') as f:
        for line in f:
            try:
                # Parse the JSON object from the line
                json_object = json.loads(line)
                
                # If the line contains loss metrics, save it in the loss dictionary
                if 'loss' in json_object and 'step' in json_object:
                    step = json_object['step']
                    loss_dict[step] = {
                        'loss': json_object['loss'],
                        'loss_cls': json_object.get('loss_cls', None),
                        'loss_bbox': json_object.get('loss_bbox', None),
                        'loss_bbox_rf': json_object.get('loss_bbox_rf', None)
                    }
                
                # If the line contains mAP metrics, save it in the mAP dictionary
                if 'coco/bbox_mAP' in json_object and 'step' in json_object:
                    step = json_object['step']
                    mAP_dict[step] = {
                        'coco/bbox_mAP': json_object['coco/bbox_mAP'],
                        'coco/bbox_mAP_50': json_object.get('coco/bbox_mAP_50', None),
                        'coco/bbox_mAP_75': json_object.get('coco/bbox_mAP_75', None)
                    }
                    
            except json.JSONDecodeError as e:
                print(f"Error in line: {e}")
    
    # Convert the dictionaries to DataFrames
    df_loss = pd.DataFrame.from_dict(loss_dict, orient='index').reset_index().rename(columns={'index': 'step'})
    df_mAP = pd.DataFrame.from_dict(mAP_dict, orient='index').reset_index().rename(columns={'index': 'step'})
    
    return df_loss, df_mAP





In [ ]:
file_path = '/home/roberto/PythonProjects/S2RAWVessel/checkpoints/MS3/vfnet_r50_fpn_1x_ms3/20230901_112855_LR_{0.0001}_BATCH_{6}/20230901_112858/vis_data/scalars.json'
# Generate the DataFrames using the function
df_loss, df_mAP = encode_metrics_to_separate_dataframes(file_path)

In [ ]:
# Importing Seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
# Set the Seaborn style to "darkgrid"
sns.set_style("darkgrid")

def plot_metric(df, column_names, title, **kwargs):
    """
    Plots a specified metric against 'step' from a DataFrame.
    
    Parameters:
        df (DataFrame): The DataFrame containing the metrics.
        column_name (str): The name of the column to plot.
        title (str): The title of the plot.
    """
    
    # log scale:
    plt.figure(figsize=(6, 5), dpi=220)
    for item in column_names:
        sns.lineplot(x='step', y=item, data=df)
    
    plt.xlabel('Step')
    if 'log' in kwargs and kwargs['log']:
        plt.yscale('log')
    if 'xlabel' in kwargs:
        plt.xlabel(kwargs['xlabel'])
    if 'ylabel' in kwargs:
        plt.ylabel(kwargs['ylabel'])
    else:
        plt.ylabel(item)
        
    if 'legend' in kwargs:
        plt.legend(kwargs['legend'])
    plt.show()

# Plotting an example: 'loss' from df_loss
specs = {'log': True}
plot_metric(df_loss, ['loss'], 'Loss over Steps', **specs)

# Plotting an example: 'coco/bbox_mAP' from df_mAP
specs = {'log': False, 'xlabel': 'Epochs', 'ylabel': '$mAP@$', 'legend':['$mAP_{50}$','$mAP_{75}$','$mAP$']}
plot_metric(df_mAP, ['coco/bbox_mAP_50','coco/bbox_mAP_75','coco/bbox_mAP'], 'mAP over Epochs', **specs)

